_General information_

_Author: Nourou-Dine YESSOUFOU_ 

_Engineer Statistician and Economist, Data scientist_

# Introduction

Hi!

I'm going to be showing here how to retrieve macroeconomic data from IMF (International Monetary Found) data warehouse into your application using their APIs. Before we start, let's keep in mind few things:

- The base link to get access to any data via API is: "http://dataservices.imf.org/REST/SDMX_JSON.svc/"
- Any data resquest to the API is in the format: "http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database ID}/{frequency}.{item1 from dimension1}+{item2 from dimension1}+{item N from dimension1}.{item1 from dimension2}+{item2 from dimension2}+{item M from dimension2}?startPeriod={start date}&endPeriod={end date}"

'CompactData' refers to a method. IMF API actually gives access to data through some methods depending on the kind of data we are looking for.

Here are available methods.

- Dataflow
- DataStructure
- CompactData
- MetadataStructure
- GenericMetadata
- CodeList
- MaxSeriesInResult

Get to know more about these methodes by clicking <a href="https://datahelp.imf.org/knowledgebase/articles/667681-json-restful-web-service">here</a>

Let's get started. Our objective is to build a link like this : http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.FR and retrive all the data available there and then more interesting things ! This link contains the annual import of Senegal from France since 1960. 

We start by exploring the diversity of data we can get access to through IMF API. This is possible thanks to "Dataflow" method.

# The Dataflow method

If you have a look at <a href="https://datahelp.imf.org/knowledgebase/articles/667681-json-restful-web-service">IMF DATA</a>, you will notice a list of databases categories which are the series availables. 

The **Dataflow** method offers JSON formatted information about which series are available through the API. To find a specoific series of interest (for example Direction of Trade Statistics DOT), you can use the **search_imf_series()** function below.

In [63]:
# Import packages
import requests
import pandas as pd

# We define a function taking a key word as argument.
def search_imf_series(search_term=''):
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    key = 'Dataflow'  # Method with series information
    series_list = requests.get(f'{url}{key}').json()['Structure']['Dataflows']['Dataflow']
    output_list = []
    # Use dict keys to navigate through results:
    for series in series_list:
        if search_term in series['Name']['#text']:
            output_list.append([series['Name']['#text'], series['KeyFamilyRef']['KeyFamilyID']])
    return pd.DataFrame(output_list, columns = ["Description", "Series Code"])

Assuming I'm looking for information about Trade. I will type "Trade" as search_term

In [64]:
## Find a database by keywords
LIST = search_imf_series("Trade")
LIST

,Description,Series Code
0,"Direction of Trade Statistics (DOTS), 2020 Q1",DOT_2020Q1
1,"Direction of Trade Statistics (DOTS), 2018 Q3",DOT_2018Q3
2,"Direction of Trade Statistics (DOTS), 2018 Q2",DOT_2018Q2
3,"Direction of Trade Statistics (DOTS), 2019 Q1",DOT_2019Q1
4,"Direction of Trade Statistics (DOTS), 2017 Q2",DOT_2017Q2
5,"Direction of Trade Statistics (DOTS), 2018 Q1",DOT_2018Q1
6,"Direction of Trade Statistics (DOTS), 2019 Q4",DOT_2019Q4
7,"Direction of Trade Statistics (DOTS), 2017 Q4",DOT_2017Q4
8,"Direction of Trade Statistics (DOTS), 2019 Q2",DOT_2019Q2
9,"Direction of Trade Statistics (DOTS), 2020 Q2",DOT_2020Q2


I have the list of all series available about trade and their corresponding code. THe code linked to the series "Direction of Trade Statistics" is "DOT". This is our **database ID** and will be useful in the next steps.

# Dimensions of a series

Now that we know the ID of our database (series containing many data tables), we need to know the format of the **key** required to get data tables from the series. 

The dimensions of the data are found with the **DataStructure** method and series specific. Use the following function to know the dimensions of "DOT" series.

In [65]:
def dimension_idf_series(series='DOT'):
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    key = f'DataStructure/{series}'  # Method / series
    key = f'DataStructure/DOT'  # Method / series
    dimension_list = requests.get(f'{url}{key}').json()['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']
    dimension_list2 = []
    for i in dimension_list:
        dimension_list2.append(i['@codelist'])
    return dimension_list2

Ci-dessous les dimensions de la série "DOT"

In [66]:
dimension_idf_series('DOT')

['CL_FREQ', 'CL_AREA_DOT', 'CL_INDICATOR_DOT', 'CL_COUNTERPART_AREA_DOT']

DOT has four dimensions. The dimensions correspond to: 1) frequency of the Data which can be **M** for monthly, **Q** for quarterly and **A** for annually, 2) first country or reference area 3) indicator (such as total exports), and 4) second country or reference area. 

For example, the key "A.SN.TXG_FOB.FR" refers to the annually value of goods exports from Senegal to France. Each dimension is seperated by a period.

# Finding the codes for each dimension

The codes which correspond to the dimensions identified above are combined, in order, and separated by periods, to complete the API request url. To find the list of possible codes for each dimension, we can use the **CodeList** method, shown below for dimension 3, indicators CL_INDICATOR_DOT.

**Below is therefore just an example for the dimension three. Same applies for the other dimensions**

In [67]:
# Example: codes for third dimension, which is 2 in python
def codeDim_imf_serie(dimension=''):
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    key = f"CodeList/{dimension}"
    print(key)
    requests.get(f'{url}{key}')
    requests.get(f'{url}{key}').json()['Structure']['CodeLists']['CodeList']['Code']
    code_list = requests.get(f'{url}{key}').json()['Structure']['CodeLists']['CodeList']['Code']
    code_list
    outlist = []
    for code in code_list:
        outlist.append([code['Description']['#text'], code['@value']])
    return pd.DataFrame(outlist)

In [68]:
print("Liste of possible values for dimension CL_AREA_DOT:\n", codeDim_imf_serie("CL_AREA_DOT"))
print("Liste of possible values for dimension CL_INDICATOR_DOT:\n", codeDim_imf_serie("CL_INDICATOR_DOT"))
print("Liste of possible values for dimension CL_COUNTERPART_AREA_DOT:\n", codeDim_imf_serie("CL_COUNTERPART_AREA_DOT"))

CodeList/CL_AREA_DOT
Liste of possible values for dimension CL_AREA_DOT:
                                                      0       1
0                                          Afghanistan      AF
1                                 Africa not allocated     F19
2                                              Albania      AL
3                                              Algeria      DZ
4                                       American Samoa      AS
..                                                 ...     ...
242  Middle East, North Africa, Afghanistan, and Pa...  1C_440
243                       Other Countries n.i.e. (IMF)     X88
244                                 Sub-Saharan Africa      F6
245                                 Western Hemisphere     A10
246                    All Countries, excluding the IO     W00

[247 rows x 2 columns]
CodeList/CL_INDICATOR_DOT
Liste of possible values for dimension CL_INDICATOR_DOT:
                                                    0          

# Final request link and data extraction

We've gathered all information needed to make the request ! Let's proceed.

In [69]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
#startdate='1960-04'
#enddate='2023-04'
# If you want to specify period
# key = f'CompactData/DOT/M.F6.TMG_CIF_USD.US/?startPeriod={startdate}&endPeriod={enddate}'
key = f'CompactData/DOT/A.SN.TMG_CIF_USD.FR' # adjust codes here
request_link = f'{url}{key}'
# Navigate to series in API-returned JSON data
data = (requests.get(request_link).json()['CompactData']['DataSet']['Series'])

In [70]:
print(request_link)

http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.FR


In [71]:
data

{'@FREQ': 'A',
 '@REF_AREA': 'SN',
 '@INDICATOR': 'TMG_CIF_USD',
 '@COUNTERPART_AREA': 'FR',
 '@UNIT_MULT': '6',
 '@TIME_FORMAT': 'P1Y',
 'Obs': [{'@TIME_PERIOD': '1960', '@OBS_VALUE': '118.2'},
  {'@TIME_PERIOD': '1961', '@OBS_VALUE': '103.1'},
  {'@TIME_PERIOD': '1962', '@OBS_VALUE': '100.6'},
  {'@TIME_PERIOD': '1963', '@OBS_VALUE': '102.5'},
  {'@TIME_PERIOD': '1964', '@OBS_VALUE': '100.8'},
  {'@TIME_PERIOD': '1965', '@OBS_VALUE': '87.4'},
  {'@TIME_PERIOD': '1966', '@OBS_VALUE': '82.2'},
  {'@TIME_PERIOD': '1967', '@OBS_VALUE': '75.543'},
  {'@TIME_PERIOD': '1968', '@OBS_VALUE': '78.8'},
  {'@TIME_PERIOD': '1969', '@OBS_VALUE': '84.261'},
  {'@TIME_PERIOD': '1970', '@OBS_VALUE': '98.996'},
  {'@TIME_PERIOD': '1971', '@OBS_VALUE': '103.309'},
  {'@TIME_PERIOD': '1972', '@OBS_VALUE': '137.083'},
  {'@TIME_PERIOD': '1973', '@OBS_VALUE': '167.5'},
  {'@TIME_PERIOD': '1974', '@OBS_VALUE': '184.225'},
  {'@TIME_PERIOD': '1975', '@OBS_VALUE': '240.981'},
  {'@TIME_PERIOD': '1976', '@OBS

# From raw to dataframe

In [72]:
import pandas as pd

data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')] for obs in data['Obs']]
df = pd.DataFrame(data_list, columns=['date', 'value'])
df["ExportCountry1"], df["ExportCountry2"]=["SN", "FR"]
df

,date,value,ExportCountry1,ExportCountry2
0,1960,118.2,SN,FR
1,1961,103.1,SN,FR
2,1962,100.6,SN,FR
3,1963,102.5,SN,FR
4,1964,100.8,SN,FR
...,...,...,...,...
58,2018,1036.293643,SN,FR
59,2019,1366.600066,SN,FR
60,2020,1222.34261,SN,FR
61,2021,1141.01514,SN,FR


So that's it!

We've got our data on annual imports of Senegal from France until now. Now let's assume we would like to have this data not only between Senegal and France but also some many other couples of countries, which is generally required for some econometric modeling. This is what we show in next section.

# Get bilateral trade data 

## Selecting the countries

In [74]:
BaseCountries1 = codeDim_imf_serie("CL_AREA_DOT")
BaseCountries2 = codeDim_imf_serie("CL_COUNTERPART_AREA_DOT")
print(BaseCountries1)
print(BaseCountries2)

CodeList/CL_AREA_DOT
CodeList/CL_COUNTERPART_AREA_DOT
                                                     0       1
0                                          Afghanistan      AF
1                                 Africa not allocated     F19
2                                              Albania      AL
3                                              Algeria      DZ
4                                       American Samoa      AS
..                                                 ...     ...
242  Middle East, North Africa, Afghanistan, and Pa...  1C_440
243                       Other Countries n.i.e. (IMF)     X88
244                                 Sub-Saharan Africa      F6
245                                 Western Hemisphere     A10
246                    All Countries, excluding the IO     W00

[247 rows x 2 columns]
                                                     0       1
0                                          Afghanistan      AF
1                                 Africa

## Importing data

In [80]:
import time
# !pip install datatable
import datatable as dt

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

## After reviewing the whole list i decided i'm interested in 5 countries bi-lateral export data
countries1=['CI', 	'SN', 	'FR',	'US',	'IN']
countries2=['CI', 	'SN', 	'FR',	'US',	'IN']

AllcountryData = pd.DataFrame()
max_retries = 2   # retry twice because of limitations in terms of number of requests per second
retry_delay = 5  # in case of failure

for i in countries1:
    for j in countries2:
        if i!=j:            
            time.sleep(5)
            key = f'CompactData/DOT/A.{i}.TMG_CIF_USD.{j}'
            print(url+key)
            
            for _ in range(max_retries):
                try:
                    data = (requests.get(f'{url}{key}').json()['CompactData']['DataSet']['Series'])
                    print("OK")
                    break
                except Exception as e:
                    print("The request didn't work:", e)
                    time.sleep(retry_delay)  # Wait before retrying

            try:
                data = (requests.get(f'{url}{key}').json()['CompactData']['DataSet']['Series'])
                data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')] for obs in data['Obs']]
                df = pd.DataFrame(data_list, columns=['date', 'value'])
                df["ExportCountry1"], df["ExportCountry2"]=[i, j]
                AllcountryData = AllcountryData.append(df)
            except Exception as e:
                print("An unknown error occurred:", e)

http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.CI.TMG_CIF_USD.SN
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.CI.TMG_CIF_USD.FR
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.CI.TMG_CIF_USD.US
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.CI.TMG_CIF_USD.IN
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.CI
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.FR
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.US
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.SN.TMG_CIF_USD.IN
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.FR.TMG_CIF_USD.CI
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.FR.TMG_CIF_USD.SN
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/DOT/A.FR.TMG_CIF_USD.US
OK
http://dataservices.imf.org/REST/SDMX_JSON.svc/Compact

## Storing data

In [81]:
AllcountryData = dt.Frame(AllcountryData)
AllcountryData.to_csv('DataImportsCIF.csv')
AllcountryData

,date,value,ExportCountry1,ExportCountry2
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1962,8.8,CI,SN
1,1963,6.3,CI,SN
2,1964,6.2,CI,SN
3,1965,3.9,CI,SN
4,1966,5.3,CI,SN
5,1967,5.7,CI,SN
6,1968,7.3,CI,SN
7,1969,7.32,CI,SN
8,1970,10.28,CI,SN
